In [7]:
import numpy as np
import sys
import os 

In [8]:
!sh del_all.sh

In [9]:
logs_path = 'slurm_logs/'
slurm_path = 'slurm_scripts/'

In [11]:
j=0
def qos(i):
    if i < 4-j:
        return 'normal', 16
    elif i < 12 -j:
        return 'm', 12
    elif i < 24-j:
        return 'm2', 8
    elif i < 44-j: 
        return 'm3', 4
    elif i < 76-j:
        return 'm4', 2
    elif i < 140-j: 
        return 'm5', 1
    elif i < 142: 
        return 'long', 48
    else:
        return 'scavenger', 96
    
def qos_deadline(i):
    if i < 12-j:
        return 'deadline', 24
    elif i < 16-j:
        return 'normal', 16
    elif i < 24-j:
        return 'm', 12
    elif i < 36-j:
        return 'm2', 8
    elif i < 56-j: 
        return 'm3', 4
    elif i < 88-j:
        return 'm4', 2
    elif i < 152-j: 
        return 'm5', 1
    else:
        return 'scavenger', 24

In [12]:
config = {
    'job_name': 'name',
    'c': 20,
    'p': 'a40,rtx6000,t4v1,t4v2',
    'gres': 'gpu:1',
    'mem': '64G',
}

In [ ]:
def write_file(name, qos, time, config, args=None, job_id=0):
    if not os.path.exists(slurm_path):
        os.mkdir(slurm_path)
    with open(os.path.join(slurm_path, '{}_{}.slrm'.format(config['job_name'], job_id)), 'w') as file:
        file.write(
            '''#!/bin/bash
#SBATCH --job-name={}_{}\n'''.format(name, job_id))
        if qos == 'deadline':
            file.write('#SBATCH --account=deadline\n')
        file.write('''#SBATCH -c {}\n'''.format(config['c']))
        file.write('''#SBATCH --qos={}\n'''.format(qos))
        file.write('''#SBATCH --partition={}\n'''.format(config['p']))
        file.write('''#SBATCH --mem={}\n'''.format(config['mem']))
        file.write('''#SBATCH --gres={}\n'''.format(config['gres']))
        file.write('''#SBATCH --time={}:00:00\n'''.format(time))
        file.write('''#SBATCH --output={}/{}_{}.out\n'''.format(slurm_path, config['job_name'], job_id))
        file.write('''#SBATCH --error={}/{}_{}.out\n'''.format(slurm_path, config['job_name'], job_id))
        file.write('''--open-mode=append\n''')
        file.write('''#SBATCH --signal=B:USR@120\n''')
        file.write('''term_handler() {\n''') 
        file.write('''\tscontrol requeue "$SLURM_JOB_ID}"\n''')
        file.write('''source ~/.bashrc\nconda activate dis\nwhich python\n''')
        file.write('''python -u run.py ''')
        for k, v in args:
            file.write('''--{}={} '''.format(k, v))
        file.write('''--slurm_job_id=$SLURM_JOB_ID &\n''')
        file.write('''child="$!"\n''')
        file.write('''wait "$child"''')


In [ ]:
write_file('test', 0, 4, config, 0)